In [93]:
import pandas as pd
import numpy as np

In [ ]:
# this notebook provides an example of the procedure used to generate (batches of) the synthetic datasets

In [ ]:
# generate records with Mistral
# prompt = 'generate a dataset of 200 fictional books, each book needs to have an id plus a few more attributes, including a plot attribute. all of them will have different titles and plots. no records with the same title or plot, please. the records should be written in languages like English, Italian, German, Spanish and French. format the dataset as a CSV'

In [176]:
data_init_df = pd.read_csv('../datasets/mistral/books/data_init.csv').reset_index(drop=True)
data_init_df['id'] = np.arange(len(data_init_df))
data_init_df.to_csv('../datasets/mistral/books/data_init.csv', index=False)

In [137]:
m = 50
p = 50
table_a_df = data_init_df.iloc[:m]
table_b_df = data_init_df.iloc[-p:]

In [138]:
table_a_df.to_csv('../datasets/mistral/books/table_a_new.csv', index=False)
table_b_df.to_csv('../datasets/mistral/books/table_b_new.csv', index=False)

In [139]:
a_matching_records_df = table_a_df.sample(n=int(len(table_b_df)/3))
a_matching_records_df.to_csv('../datasets/mistral/books/matching_records_new.csv', index=False)

In [ ]:
# alter the records with Mistral
# prompt = 'given the following dataset, generate an alternative textual representation for each record such that they still represent the same entity:' + str(matching_records_new.csv) + 'return the dataset as a csv'

In [140]:
a_matching_records_df = pd.read_csv('../datasets/mistral/books/matching_records_altered.csv')
table_b_new_df = pd.concat([table_b_df, a_matching_records_df], axis=0)
table_b_new_df.head()

,id,title,author,language,genre,plot
53,964,Las Llamas Congeladas,Isabella Rodríguez,Spanish,Mystery,Un científico debe encontrar una manera de der...
54,965,Les Flammes Gelées,Sophie Martin,French,Romance,Un scientifique doit trouver un moyen de faire...
55,966,The Shadow Weaver,Mia Thompson,English,Fantasy,In a land where shadows can be woven into tape...
56,967,Il Tessitore delle Ombre,Luca Bianchi,Italian,Adventure,Un giovane tessitore deve creare un arazzo abb...
57,968,Der Schattenweber,Eva Müller,German,Mystery,Ein junger Weber muss einen Wandteppich erscha...


In [148]:
train_df = pd.read_csv('../datasets/mistral/books/train.csv')
test_df = pd.read_csv('../datasets/mistral/books/test.csv')

In [149]:
matching_pairs_df = a_matching_records_df.sample(n=int(len(a_matching_records_df)/3))['id']
sample_size = int(len(matching_pairs_df)/2)
new_train_df = pd.concat([matching_pairs_df[:sample_size], matching_pairs_df[:sample_size]], axis=1)
new_train_df['label'] = 1
new_train_df.columns = ['ltable_id', 'rtable_id', 'label']

new_test_df = pd.concat([matching_pairs_df[-sample_size:], matching_pairs_df[-sample_size:]], axis=1)
new_test_df['label'] = 1
new_test_df.columns = ['ltable_id', 'rtable_id', 'label']

In [167]:
train_df = pd.concat([new_train_df, train_df], axis=0)

In [168]:
test_df = pd.concat([new_test_df, test_df], axis=0)

In [166]:
left_non_match = table_a_df.sample(n=int(sample_size/2))['id']
left_non_match = pd.DataFrame(left_non_match).rename(columns={'id': "ltable_id"}).reset_index(drop=True)

right_non_match = table_b_df.sample(n=int(sample_size/2))['id']
right_non_match = pd.DataFrame(right_non_match).rename(columns={'id': "rtable_id"}).reset_index(drop=True)

non_matching_pairs_df = pd.concat([left_non_match, right_non_match], axis=1)
non_matching_pairs_df['label'] = 0

In [162]:
test_df = pd.concat([test_df, non_matching_pairs_df[:int(len(non_matching_pairs_df)/2)]], axis=0, ignore_index=True)
train_df = pd.concat([train_df, non_matching_pairs_df[-int(len(non_matching_pairs_df)/2):]], axis=0, ignore_index=True)

In [163]:
train_df.to_csv('../datasets/mistral/books/train.csv', index=False)
test_df.to_csv('../datasets/mistral/books/test.csv', index=False)